In [ ]:
import mysql.connector

# Connection details
host = 'localhost'
user = 'root'
password = ''
database = 'data_smartphone'
port = 3306

# Establish a connection to the database
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    port=port
)

print("Koneksi ke database berhasil")


In [ ]:
import pandas as pd
from IPython.display import display

def execute_query_to_dataframe(connection, query):
    try:
        # Create a cursor object
        cursor = connection.cursor(dictionary=True)
        
        # Execute the SQL query
        cursor.execute(query)
        
        # Fetch all the results
        result = cursor.fetchall()
        
        # Convert the results to a Pandas DataFrame
        df = pd.DataFrame(result)
        
        return df
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        # Close the cursor but not the connection
        cursor.close()

# Define the SQL query for LEFT JOIN
query = """
SELECT 
    data.Merek,
    data.Tipe,
    data.Bulan,
    data.Jumlah_Stok,
    data.Jumlah_Terjual,
    data.Harga_Satuan_Rp,
    data.Total_Penjualan_Rp,
    data_spesifikasi.Kamera_Utama_MP,
    data_spesifikasi.Kamera_Depan_MP,
    data_spesifikasi.RAM,
    data_spesifikasi.Memori_Internal,
    data_spesifikasi.Baterai_mAh,
    data_spesifikasi.Jenis_Layar
FROM 
    data
LEFT JOIN 
    data_spesifikasi 
ON 
    data.Tipe = data_spesifikasi.Tipe;
"""

# Fetch the data and convert to DataFrame if connection is successful
if connection.is_connected():
    df = execute_query_to_dataframe(connection, query)
    if df is not None and not df.empty:
        display(df.head())
    else:
        print("No data found or query execution failed.")
else:
    print("No connection to the database.")


In [ ]:
# Persiapan data: jumlah terjual per bulan
df_klastering = df[['Bulan', 'Jumlah_Terjual']].copy()

# Pastikan data yang digunakan adalah numerik
df_klastering['Jumlah_Terjual'] = pd.to_numeric(df_klastering['Jumlah_Terjual'], errors='coerce')
df_klastering = df_klastering.dropna()

print("Data untuk klastering:")
display(df_klastering.head())


In [ ]:
from sklearn.cluster import Birch

# Inisialisasi model BIRCH
birch_model = Birch(n_clusters=None, threshold=0.5)

# Fit model BIRCH ke data
birch_model.fit(df_klastering[['Jumlah_Terjual']])

# Prediksi klaster
clusters = birch_model.predict(df_klastering[['Jumlah_Terjual']])

print("Klaster hasil prediksi:")
print(clusters)


In [ ]:
import pandas as pd

# Fungsi untuk mengklasifikasikan penjualan
def classify_sales(row):
    if row['Jumlah_Terjual'] <= 2:
        return 'Sangat Rendah'
    elif row['Jumlah_Terjual'] <= 5:
        return 'Rendah'
    elif row['Jumlah_Terjual'] <= 10:
        return 'Cukup'
    elif row['Jumlah_Terjual'] <= 20:
        return 'Berpotensi Tinggi'
    elif row['Jumlah_Terjual'] <= 50:
        return 'Tinggi'
    else:
        return 'Sangat Tinggi'

# Asumsi df sudah diisi dengan data yang relevan
df['Kategori_Penjualan'] = df.apply(classify_sales, axis=1)

# Menampilkan DataFrame dengan kategori penjualan
print("DataFrame dengan kategori penjualan:")
display(df)

# Menyimpan DataFrame ke dalam file Excel
file_path = 'hasil_klastering.xlsx'
df.to_excel(file_path, index=False)
print(f"DataFrame telah disimpan ke {file_path}")


In [ ]:
# Menghitung jumlah dan persentase distribusi
distribution_counts = df['Kategori_Penjualan'].value_counts()
distribution_percentages = df['Kategori_Penjualan'].value_counts(normalize=True) * 100

# Membuat DataFrame untuk distribusi
distribution_df = pd.DataFrame({
    'Jumlah Tipe': distribution_counts,
    'Persentase (%)': distribution_percentages
})

# Menampilkan tabel distribusi
print("Distribusi Tipe Smartphone Berdasarkan Kategori Tingkat Penjualan:")
display(distribution_df)




In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Menghitung jumlah penjualan setiap merek
penjualan_merek = df.groupby('Merek')['Jumlah_Terjual'].sum().sort_values(ascending=False)

# Menghasilkan warna yang berbeda untuk setiap merek
colors = plt.cm.tab10(np.arange(len(penjualan_merek)))

# Membuat diagram batang dengan warna yang berbeda
plt.figure(figsize=(12, 6))
penjualan_merek.plot(kind='bar', color=colors)
plt.title('Total Penjualan Berdasarkan Merek Smartphone')
plt.xlabel('Merek')
plt.ylabel('Total Penjualan')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Menampilkan legenda untuk warna setiap merek
patches = [plt.Rectangle((0,0),1,1, color=colors[i], label=merek) for i, merek in enumerate(penjualan_merek.index)]
plt.legend(handles=patches, loc='upper right')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Definisi batasan untuk setiap kelas
class_boundaries = {
    'Sangat Rendah': (0, 2),
    'Rendah': (3, 5),
    'Cukup': (6, 10),
    'Berpotensi Tinggi': (11, 20),
    'Tinggi': (21, 50),
    'Sangat Tinggi': (51, np.inf)
}

# Warna untuk setiap kategori penjualan
class_colors = {
    'Sangat Rendah': 'blue',
    'Rendah': 'green',
    'Cukup': 'yellow',
    'Berpotensi Tinggi': 'orange',
    'Tinggi': 'red',
    'Sangat Tinggi': 'purple'
}

# Visualisasi hasil klastering berdasarkan kategori penjualan
plt.figure(figsize=(14, 8))

# Plot area arsiran untuk setiap kelas
for class_name, (lower, upper) in class_boundaries.items():
    plt.fill_betweenx(y=[lower, upper], x1=0, x2=len(df), color=class_colors[class_name], alpha=0.2, label=f'{class_name}')

# Plot data penjualan berdasarkan kategori penjualan
for idx, row in df.iterrows():
    plt.scatter(idx, row['Jumlah_Terjual'], color=class_colors[row['Kategori_Penjualan']], label=row['Kategori_Penjualan'])

plt.title('Distribusi Penjualan Berdasarkan Kategori Klastering Menggunakan BIRCH')
plt.xlabel('Indeks Data')
plt.ylabel('Jumlah Terjual')
plt.grid(True)

# Menambahkan legend untuk keterangan warna berdasarkan kategori penjualan
handles = [plt.Rectangle((0,0),1,1, color=class_colors[label]) for label in class_boundaries.keys()]
labels = class_boundaries.keys()
plt.legend(handles, labels, loc='upper left')

# Fungsi untuk menampilkan merek dan tipe saat scatter plot diklik
def on_click(event):
    if event.xdata is not None and event.ydata is not None:
        idx = int(event.xdata)
        plt.gca().set_title('')
        plt.gca().annotate(f'Merek: {df.iloc[idx]["Merek"]}, Tipe: {df.iloc[idx]["Tipe"]}', xy=(event.xdata, event.ydata), xytext=(event.xdata + 10, event.ydata + 5), bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=0.5), arrowprops=dict(arrowstyle="->", lw=0.5))

plt.gcf().canvas.mpl_connect('button_press_event', on_click)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Definisikan urutan bulan yang benar
order_of_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

# Pastikan kolom 'Bulan' di DataFrame adalah tipe kategori dengan urutan yang benar
df['Bulan'] = pd.Categorical(df['Bulan'], categories=order_of_months, ordered=True)

# Menghitung jumlah penjualan setiap merek per bulan
penjualan_bulanan = df.pivot_table(
    index='Merek',
    columns='Bulan',
    values='Jumlah_Terjual',
    aggfunc='sum',
    fill_value=0
)

# Menggunakan heatmap dengan seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(penjualan_bulanan, annot=True, cmap='YlGnBu', fmt='d')
plt.title('Heatmap Distribusi Penjualan Berdasarkan Merek Smartphone')
plt.xlabel('Bulan')
plt.ylabel('Merek')
plt.yticks(rotation=0)
plt.show()


In [ ]:

# Definisikan batasan untuk setiap kelas
class_boundaries = {
    'Sangat Rendah': (0, 2),
    'Rendah': (3, 5),
    'Cukup': (6, 10),
    'Berpotensi Tinggi': (11, 20),
    'Tinggi': (21, 50),
    'Sangat Tinggi': (51, np.inf)
}

# Warna untuk setiap kelas
class_colors = {
    'Sangat Rendah': 'blue',
    'Rendah': 'green',
    'Cukup': 'yellow',
    'Berpotensi Tinggi': 'orange',
    'Tinggi': 'red',
    'Sangat Tinggi': 'purple'
}

# Visualisasi hasil klastering berdasarkan kategori penjualan
plt.figure(figsize=(14, 8))

# Plot area arsiran untuk setiap kelas
for class_name, (lower, upper) in class_boundaries.items():
    plt.fill_betweenx(y=[lower, upper], x1=0, x2=len(df), color=class_colors[class_name], alpha=0.2, label=f'{class_name} (area)')

# Plot data penjualan
for class_name in df['Kategori_Penjualan'].unique():
    subset = df[df['Kategori_Penjualan'] == class_name]
    plt.scatter(range(len(subset)), subset['Jumlah_Terjual'], label=class_name, color=class_colors[class_name])

plt.title('Distribusi Penjualan Berdasarkan Kategori Klastering Menggunakan BIRCH')
plt.xlabel('Indeks Data')
plt.ylabel('Jumlah Terjual')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Membuat pivot table untuk merangkum data penjualan bulanan per tipe dan merek
penjualan_bulanan = df.pivot_table(
    index=['Merek', 'Tipe'],
    columns='Bulan',
    values='Jumlah_Terjual',
    aggfunc='sum'
).reset_index()

# Mengisi nilai NaN dengan 0 (jika ada bulan yang tidak ada penjualan)
penjualan_bulanan = penjualan_bulanan.fillna(0)

# Urutkan kolom bulan dengan urutan yang benar
bulan_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
penjualan_bulanan = penjualan_bulanan[['Merek', 'Tipe'] + bulan_order]

# Mengklasifikasikan penjualan untuk setiap bulan
for bulan in bulan_order:
    penjualan_bulanan[bulan] = penjualan_bulanan[bulan].apply(classify_sales)

# Menampilkan tabel hasil
print("Status Penjualan Setiap Tipe dan Merek Selama 6 Bulan:")
display(penjualan_bulanan)

# Menyimpan hasil ke dalam file Excel
file_path = 'penjualan_bulanan_klasifikasi.xlsx'
penjualan_bulanan.to_excel(file_path, index=False)
print(f"DataFrame telah disimpan ke {file_path}")
